# Landsat 8 OLI TIRS Reflectancia Superficie google Earth Engine

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [3]:
# Agregar un shp GEE
ee_zona = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [4]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='SATELLITE')

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP'

In [5]:
# Visualizar en mapa
geometria = ee_zona.geometry()
Map.centerObject(geometria, 11)
Map.addLayer(ee_zona, {"color" : "00FF11"}, name = "Zona")
Map

## Coleccion de Landsat 8 OLI TIRS RS

In [8]:
L8_RS = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
          .filterDate('2019-01-01','2019-12-31')\
          .filterMetadata('CLOUD_COVER','less_than',20)\
          .filterBounds(ee_zona)

In [9]:
# Conteo de imagenes
count = L8_RS.size().getInfo()
print("Cantidad de imagenes L8_RS:", count)

Cantidad de imagenes L8_RS: 4


## Obtener ID, Fecha, nubosidad, Path y Row

In [10]:
# Imprimir la lista de ID Imagenes
ID_L8 = L8_RS.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(ID_L8)

['LC08_003069_20190621', 'LC08_003069_20190808', 'LC08_003069_20190824', 'LC08_003069_20190925']


In [11]:
def list_coll(coll):
    def func_ejv(im):
        return ee.String(ee.Image(im).date().format('YYYY/MM/dd')).slice(0)
    return coll.toList(coll.size(), 0).map(func_ejv)

In [12]:
fecha = list_coll(L8_RS).getInfo()
print(fecha)

['2019/06/21', '2019/08/08', '2019/08/24', '2019/09/25']


In [13]:
# Imprimir la lista Porcentaje Nubosidad
porcentaje_nube = L8_RS.reduceColumns(ee.Reducer.toList(),["CLOUD_COVER"]).get("list").getInfo()
print(porcentaje_nube)

[11.9, 1.1, 17.97, 5.46]


In [14]:
# Imprimir la lista Path y Row
WRS_PATH = L8_RS.reduceColumns(ee.Reducer.toList(),["WRS_PATH"]).get("list").getInfo()
WRS_ROW = L8_RS.reduceColumns(ee.Reducer.toList(),["WRS_ROW"]).get("list").getInfo()
print(WRS_PATH)
print(WRS_ROW)

[3, 3, 3, 3]
[69, 69, 69, 69]


In [15]:
# Crear un diccionario con las listas
dic = {"ID": ID_L8, "Fecha": fecha,"Porcentaje Nube": porcentaje_nube,"PATH": WRS_PATH,"ROW":WRS_ROW}

In [16]:
import pandas as pd

In [17]:
# Crear un DataFrame mediante el diccionario
df1 = pd.DataFrame(data = dic)
print(df1)
print("Tipo de clase: ", type(dic))

                     ID       Fecha  Porcentaje Nube  PATH  ROW
0  LC08_003069_20190621  2019/06/21            11.90     3   69
1  LC08_003069_20190808  2019/08/08             1.10     3   69
2  LC08_003069_20190824  2019/08/24            17.97     3   69
3  LC08_003069_20190925  2019/09/25             5.46     3   69
Tipo de clase:  <class 'dict'>


## Exportar tabla ID, Fecha, Porcentaje Nubosidad

In [ ]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

In [ ]:
df1.to_csv("file_name.csv") # relative position

## Seleccion imagen Landsat 8

In [20]:
ID_L8[1]

'LC08_003069_20190808'

In [18]:
# Importar imagen Landsat 8 RS 
image = ee.Image("LANDSAT/LC08/C01/T1_SR" + "/" + ID_L8[1])

In [13]:
# Nombre de las bandas
print(image.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa']


## Seleccionar Bandas

In [14]:
# Crear nombre por bandas
bandas_RS = ["B2","B3","B4","B5","B6","B7"]
bandas_TB = ["B10","B11"]

In [15]:
# Seleccionar imagenes por bandas
img_L8_RS = image.select(bandas_RS)
img_L8_TB = image.select(bandas_TB)

In [16]:
# Nombre de las bandas
print("Banda RS:",img_L8_RS.bandNames().getInfo())
print("Banda TB:",img_L8_TB.bandNames().getInfo())

Banda RS: ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
Banda TB: ['B10', 'B11']


## Verificar proyeccion

In [17]:
# Consultar Proyeccion
print("Proyeccion RS:",img_L8_RS.projection().getInfo())
print("Proyeccion TB:",img_L8_TB.projection().getInfo())

Proyeccion RS: {'type': 'Projection', 'crs': 'EPSG:32619', 'transform': [30, 0, 228285, 0, -30, -1323285]}
Proyeccion TB: {'type': 'Projection', 'crs': 'EPSG:32619', 'transform': [30, 0, 228285, 0, -30, -1323285]}


## Metadato landsat

In [18]:
# Nombres del Metadato
img_L8_RS.propertyNames().getInfo()

['IMAGE_QUALITY_TIRS',
 'CLOUD_COVER',
 'system:id',
 'EARTH_SUN_DISTANCE',
 'LANDSAT_ID',
 'system:footprint',
 'system:version',
 'CLOUD_COVER_LAND',
 'GEOMETRIC_RMSE_MODEL',
 'SR_APP_VERSION',
 'SATELLITE',
 'SOLAR_AZIMUTH_ANGLE',
 'IMAGE_QUALITY_OLI',
 'system:time_end',
 'WRS_PATH',
 'system:time_start',
 'SENSING_TIME',
 'ESPA_VERSION',
 'SOLAR_ZENITH_ANGLE',
 'WRS_ROW',
 'GEOMETRIC_RMSE_MODEL_Y',
 'LEVEL1_PRODUCTION_DATE',
 'GEOMETRIC_RMSE_MODEL_X',
 'system:asset_size',
 'PIXEL_QA_VERSION',
 'system:index',
 'system:bands',
 'system:band_names']

In [19]:
# Extraer valor metadato distancia del sol a tierra
d = img_L8_RS.get("EARTH_SUN_DISTANCE").getInfo()
d

1.014033

In [20]:
# Extraer angulo zenital solar
SOLAR_ZENITH_ANGLE = img_L8_RS.get("SOLAR_ZENITH_ANGLE").getInfo()
SOLAR_ZENITH_ANGLE

41.700932

In [21]:
# Determinacion angulo elevacion solar radianes
import math
SUN_ELEVATION = 90 - SOLAR_ZENITH_ANGLE
SUN_ELEVATION_R = SUN_ELEVATION*math.pi/180
print("Elevacion solar radianes:", SUN_ELEVATION_R)

Elevacion solar radianes: 0.8429777622446325


## Visualizar Landsat

In [26]:
# Simbologia estala 0 - 10000
viz_RS = {
    'bands': ['B6', 'B5', 'B4'],
    'min': 250,
    'max': 7000,
    'gamma': 1.90
}

In [23]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='SATELLITE')

In [24]:
# Convertir en Geometria
geometria = ee_zona.geometry()
Map.centerObject(geometria,8)

In [27]:
# Visualizar ESCALADO 0 - 10000
Map.addLayer(img_L8_RS, viz_RS,name = "Landsat8 RS")
Map

Map(bottom=35438.0, center=[-12.908744339809369, -69.99471735110599], controls=(WidgetControl(options=['positi…

## Factor de escala Reflectancia Superficie

In [28]:
# Escalar la imagen Landsat 5 RS 0 - 1
img_L8_RS_Es = img_L8_RS.multiply(0.0001)

In [29]:
# Simbologia estala 0 - 1
viz_es = {
    'bands': ['B6', 'B5', 'B4'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [30]:
# Visualizar ESCALADO 0 - 1
Map.addLayer(img_L8_RS_Es, viz_es,name = "Landsat8 RS Escalado")
Map

Map(bottom=35438.0, center=[-12.908744339809369, -69.99471735110599], controls=(WidgetControl(options=['positi…

## Factor de escala Temperatura Brillo

In [31]:
# Escalar la imagen Landsat 8 TB convertir Kelvin a Celsius
img_L8_TB_es = img_L8_TB.subtract(273.15).multiply(0.01)

In [32]:
# Crear su simbologia TB
viz_tb = {
    'palette': ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                'ff0000', 'de0101', 'c21301', 'a71001', '911003'],
    'min': 25,
    'max': 28
}

In [33]:
# Visualizacion temperatura brillo
Map.addLayer(img_L8_TB_es.select(["B10"]), viz_tb,name = "TB B10 Celsius")
Map

Map(bottom=35457.0, center=[-13.009909963906498, -70.03234863281251], controls=(WidgetControl(options=['positi…

## Reproyectar y recortar segun zona

In [34]:
# Reproyectar segun zona de estudio
img_L8_TB_es = img_L8_TB_es.reproject(crs="EPSG:32719", scale=30)
img_L8_RS_Es = img_L8_RS_Es.reproject(crs="EPSG:32719", scale=30)

In [35]:
# Recortar RS y TB
img_L8_TB_es_clip = img_L8_TB_es.clip(ee_zona)
img_L8_RS_Es_clip = img_L8_RS_Es.clip(ee_zona)

In [36]:
# Consultar Proyeccion
print("Proyeccion TB:",img_L8_TB_es_clip.projection().getInfo())
print("Proyeccion RS:",img_L8_RS_Es_clip.projection().getInfo())

Proyeccion TB: {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion RS: {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}


In [37]:
img_L8_RS_Es_clip.bandNames().getInfo()

['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

In [38]:
# Visualizacion TB y RS
Map.addLayer(img_L8_TB_es_clip.select(["B10"]), viz_tb,name = "TB B10 clip")
Map.addLayer(img_L8_RS_Es_clip, viz_es,name = "L8 RS clip")
Map

Map(bottom=35457.0, center=[-13.009909963906498, -70.03234863281251], controls=(WidgetControl(options=['positi…

## Exportar Imagen calibrado

In [39]:
os.getcwd()

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP'

In [40]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [41]:
# Exportar imagen Temperatura de brillo B10 y B11
geemap.ee_export_image(img_L8_TB_es_clip, filename="L8_TB_es_clip.tif", scale=30, region=geometria, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE


In [42]:
# Exportar imagen Reflectancia superficie B654
geemap.ee_export_image(img_L8_RS_Es_clip.select(["B6","B5","B4"]), filename="L8_RS_Es_clip.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_RS_Es_clip.tif


In [43]:
# Exportar en google drive
geemap.ee_export_image_to_drive(img_L8_RS_Es_clip, description='L8_RS_Es_clip', folder='GEE_Python', region=geometria, scale=30)

Exporting L8_RS_Es_clip ...
